First scrape the snow sensor data

In [1]:
import requests, re
import pickle as pkl
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
def get_soup(url):
    response = requests.get(url)
    page = response.text
    if response.status_code != 200:
        print("REQUESTS error")
        print(response.status_code)
    return BeautifulSoup(page, "lxml")

def str2floatflag(s):
    s = s.strip()
    letters = ''
    try:
        num = float(s)
    except ValueError:
        # return tuple contain int and str elements
        if '-' not in s:
            rs = r"[0-9|.]"
            num = "".join(re.findall(rs,s))
            letters = s.strip(num)
        else:
            num = np.NaN
    return (float(num), letters)

In [3]:
from datetime import timedelta
from datetime import date

start_date, end_date = pd.to_datetime('20170101'), pd.to_datetime('20170302')
sensor_urls = []
sensor_url_root = "https://cdec.water.ca.gov/cgi-progs/snow/PAGE6."
for d in pd.date_range(start_date, end_date):
    sensor_urls.append(sensor_url_root + date.strftime(d, "%Y%m%d"))

In [4]:
from time import sleep

snow_df = pd.DataFrame()

for url in sensor_urls:
    # get page
    sensor_soup = get_soup(url)

    # get each table row
    data_table = sensor_soup.find_all("table")[0]
    rows = [row for row in data_table.find_all('tr')]

    # get page info
    page_date = pd.to_datetime(sensor_soup.find("h2").next_element)
    print(page_date)

    # get all data for a given day, create a dataframe
    df_rows = [] # list of dicts
    df_row = {}
    for r in rows:
        if r.find_all('font'): # it's a heading
            df_row['area'] = r.find_all('font')[0].text.lower()
        elif r.find_all('td')[0].text.strip() != 'Station': # it's not, and it's not a station heading
            r_data = r.find_all('td')
            df_row['st_name'], df_row['st_code'] = r_data[0].text.strip().lower(), r_data[1].text.strip().lower()
            df_row['agency'] = r_data[2].text.strip().lower()
            df_row['elev_ft'] = int(r_data[3].text.replace(',',''))
            df_row['apr1avg_in'], df_row['apr1avg_in_code'] = str2floatflag(r_data[4].text)
            df_row['today_in'], df_row['today_in_code'] = str2floatflag(r_data[5].text)
            df_row['apr1_pct'], df_row['apr1_pct_code'] = str2floatflag(r_data[6].text.strip('%'))
            df_row['yesday_in'], df_row['yesday_in_code'] = str2floatflag(r_data[7].text)
            df_row['date'], df_row['url'] = page_date, url
            df_rows.append(df_row.copy())
    page_df = pd.DataFrame.from_dict(df_rows)
    snow_df = snow_df.append(page_df)
    print("sleep 1.5 sec")
    sleep(1.5)

2017-01-01 00:00:00
sleep 1.5 sec
2017-01-02 00:00:00
sleep 1.5 sec


KeyboardInterrupt: 

In [ ]:
# these are empty for now - should maybe make this conditional
del snow_df['apr1_pct_code']
del snow_df['apr1avg_in_code']

In [19]:
snow_df.head()

,agency,apr1_pct,apr1avg_in,area,date,elev_ft,st_code,st_name,today_in,today_in_code,url,yesday_in,yesday_in_code
0,dwr,26.0,29.2,trinity river,2017-01-01,7150,pet,peterson flat,7.8,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,7.8,r
1,dwr,43.0,39.6,trinity river,2017-01-01,6700,rrm,red rock mountain,17.1,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,17.2,r
2,dwr,NaN,40.5,trinity river,2017-01-01,6450,bnk,bonanza king,NaN,,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,NaN,
3,dwr,27.0,40.3,trinity river,2017-01-01,6400,shm,shimmy lake,11.1,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,11.3,r
4,dwr,28.0,28.3,trinity river,2017-01-01,6200,mb3,middle boulder 3,8.0,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,7.8,r


Get lat/long

In [20]:
soup = get_soup('http://cdec.water.ca.gov/misc/SnowSensors.html')

data_table = soup.find_all("table")[0]
rows = [row for row in data_table.find_all('tr')]

stations_list = []
station = {}
for r in rows:
    row_array = tuple(r.text.split('\n'))
    if len(row_array) == 7:
        station['st_code'] = row_array[0].strip().lower()
        station['st_lat'], station['st_lon'] = float(row_array[3].split()[0]), float(row_array[3].split()[1])
        stations_list.append(station.copy())

In [21]:
station_coords = pd.DataFrame.from_dict(stations_list)

In [22]:
snow_coords_df = snow_df.merge(station_coords, on='st_code', how='left')

In [23]:
snow_coords_df.head(10)

,agency,apr1_pct,apr1avg_in,area,date,elev_ft,st_code,st_name,today_in,today_in_code,url,yesday_in,yesday_in_code,st_lat,st_lon
0,dwr,26.0,29.2,trinity river,2017-01-01,7150,pet,peterson flat,7.8,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,7.8,r,41.302,-122.528
1,dwr,43.0,39.6,trinity river,2017-01-01,6700,rrm,red rock mountain,17.1,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,17.2,r,41.023,-122.885
2,dwr,NaN,40.5,trinity river,2017-01-01,6450,bnk,bonanza king,NaN,,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,NaN,,41.083,-122.628
3,dwr,27.0,40.3,trinity river,2017-01-01,6400,shm,shimmy lake,11.1,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,11.3,r,41.008,-122.800
4,dwr,28.0,28.3,trinity river,2017-01-01,6200,mb3,middle boulder 3,8.0,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,7.8,r,41.225,-122.811
5,dwr,29.0,29.9,trinity river,2017-01-01,6030,hig,highland lakes,8.9,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,8.8,r,41.093,-122.483
6,dwr,21.0,16.0,trinity river,2017-01-01,5900,sct,scott mountain,3.5,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,3.4,r,41.272,-122.718
7,dwr,28.0,22.4,trinity river,2017-01-01,5650,mum,mumbo basin,6.3,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,6.0,r,41.197,-122.523
8,dwr,37.0,15.8,trinity river,2017-01-01,5100,bfl,big flat,5.9,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,6.0,r,41.080,-122.942
9,nrcs,NaN,NaN,trinity river,2017-01-01,5100,cwf,crowder flat,0.9,r,https://cdec.water.ca.gov/cgi-progs/snow/PAGE6...,0.9,r,41.893,-120.752


Picklin' time

In [24]:
pkl.dump(snow_coords_df, open( "snow_coords_df.pkl", "wb" ) )